In [1]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [2]:
pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.72
    Uninstalling langchain-core-0.3.72:
      Successfully uninstalled langchain-core-0.3.72


In [3]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 44.3 MB/s eta 0:00:00


In [ ]:
# 1. Imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
import os

# 2. Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 3. Load and clean text
loader = TextLoader("ExamGenerator.txt")
documents = loader.load()

def clean_text(text):
    import re
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return text.lower()

for doc in documents:
    doc.page_content = clean_text(doc.page_content)

# 4. Split the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# 5. Create vectorstore
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding)

# 6. Create LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 7. Define the Exam Creator Tool
def exam_creator(topic: str) -> str:
    """Generates a multiple choice exam question from a given topic."""
    related_docs = vectorstore.similarity_search(topic, k=3)
    context = "\n".join(doc.page_content for doc in related_docs)

    prompt = f"""You are an Exam Generator AI.
Use the provided context to create 2 multiple-choice question.
Each question should have:
- 1 question
- 4 options: A, B, C, D
- Indicate the correct answer clearly.

Context:
{context}

Topic: {topic}

Generate now:
"""
    response = llm.invoke(prompt)
    return response.content

exam_tool = Tool(
    name="exam_creator",
    description="Use this tool to create multiple choice exam questions from a given topic.",
    func=exam_creator
)

# 8. Initialize the Agent
agent = initialize_agent(
    tools=[exam_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 9. Interactive session
print("\nExam Creator Agent is Ready! Type 'exit' to stop.\n")

while True:
    user_input = input("Enter a topic: ")
    if user_input.lower() == "exit":
        break

    response = agent.run(f"Create an exam question about {user_input}")
    print("\nGenerated Exam Question:\n")
    print(response)
    print("\n" + "-"*60 + "\n")


/tmp/ipython-input-1521773202.py:69: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(



Exam Creator Agent is Ready! Type 'exit' to stop.

Enter a topic: Math


/tmp/ipython-input-1521773202.py:84: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(f"Create an exam question about {user_input}")




> Entering new AgentExecutor chain...
I need to create a multiple choice exam question related to Math. 
Action: exam_creator
Action Input: "Math"
Observation: **Question 1:**  
What type of classification task involves distinguishing between two classes, such as spam and not spam?  
A) Multi-class classification  
B) Multi-label classification  
C) Binary classification  
D) Imbalanced classification  
**Correct Answer:** C) Binary classification  

---

**Question 2:**  
Which classification algorithm is known for splitting data into branches to make predictions and is easy to interpret?  
A) Logistic regression  
B) Decision trees  
C) Random forests  
D) Support vector machines  
**Correct Answer:** B) Decision trees  
Thought:I need to create a multiple choice exam question related to Math. 
Action: exam_creator
Action Input: "Math"
Observation: **Question 1:**  
What type of classification task involves distinguishing between two classes, such as spam and not spam?  
A) Multi-c